In [7]:
import numpy as np
import tensorflow.keras as keras
import tensorflow.keras.utils
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import os.path
import sklearn.model_selection
import importlib
import sys
%load_ext tensorboard

function_dir = os.path.join(os.pardir, 'Functions')
sys.path.append(function_dir)

import models as mod
importlib.reload(mod) #to make sure the last version of seg is used
import DLtools as tools
import json
import os

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
#address of the base folder:
base = "R:\\3.Masters_projects\\2021_Dominic_Rebindaine\\ROI"
#extract the name from the file name:
name_l =[]
path_l = []
for fname in os.listdir(base):
    pathto = os.path.join(base,fname)
    
    if os.path.isdir(pathto):
        path_l.append(pathto)
        split = fname.split(sep='_')
        if len(split)>2:
            name = split[1]
            name_l.append(name)
#group identical labels together:
label_name, label = np.unique(name_l,return_inverse=True)

In [4]:
#define the input data:
size = 200

In [5]:
#open the images and find their label
def data_opener(name):
    img = tf.io.decode_png(tf.io.read_file(name))
    X = tf.image.resize(img,(size,size))
    Xn = tf.image.per_image_standardization(X)
    part = tf.strings.split(name,os.sep)
    fname = part[4]
    fpart = tf.strings.split(fname,'_')
    if len(fpart)<2:
        y = -1
    else:
        el_label = fpart[1]
        y = -1
        for idx, label in enumerate(label_name):
            if label == el_label:
                y = idx
    data = (Xn,y)
    return data
def upsample(name_list, n_cat):
    n_sample = np.zeros(n_cat)
    for name in name_list:
        part = tf.strings.split(name,os.sep)
        fname = part[4]
        fpart = tf.strings.split(fname,'_')
        if len(fpart)<2:
            y = -1
        else:
            el_label = fpart[1]
            y = -1
            for idx, label in enumerate(label_name):
                if label == el_label:
                    y = idx
        data = (Xn,y)
#logs: aug:   no gaussian noise
#      aug2:  gaussian noise with stddev = 0.1
#      aug3:  gaussian noise with stddev in [0:0.2] (and no random sat or contrast)
#      aug4:  data shuffled each epoch
#      aug5:  gaussian noise removed
def data_aug(img,label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    rnd = np.random.randint(4)
    img = tf.image.rot90(img,k=rnd)
    img = tf.image.random_hue(img,0.1)
    #stddev = tf.random.uniform(shape=[1],maxval=0.05)
    #img+= tf.random.normal((size,size,3),stddev=stddev)
    return (img,label)


def input_pipeline(ds, augmentation = False):
    ds = ds.shuffle(10000)
    ds = ds.map(data_opener)
    #filter out the unlabeled elements
    ds = ds.filter(lambda X,y: y>=0)
    if augmentation:
        ds = ds.map(data_aug)
    ds = ds.batch(32)
    ds = ds.prefetch(3)
    return ds

    

In [6]:
file_names = tf.data.Dataset.list_files('R:\\3.Masters_projects\\2021_Dominic_Rebindaine\\ROI\\*\\*\\*.png')
#split the dataset now, as the loading of the image would need too long to use the list function
train, val = sklearn.model_selection.train_test_split(list(file_names),test_size= 0.2)
dt = tf.data.Dataset.from_tensor_slices(train)
dv = tf.data.Dataset.from_tensor_slices(val)

dt = input_pipeline(dt,augmentation = True)
dv = input_pipeline(dv)

In [10]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ.pop('TF_CONFIG', None)
if '.' not in sys.path:
    sys.path.insert(0, '.')


In [14]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456','localhost:34567']
    },
    'task': {'type': 'worker', 'index': 0}
}
json.dumps(tf_config)


'{"cluster": {"worker": ["localhost:12345", "localhost:23456", "localhost:34567"]}, "task": {"type": "worker", "index": 0}}'

In [13]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0',), communication = CommunicationImplementation.AUTO


In [ ]:
with strategy.scope():
     #Model building/compiling need to be within `strategy.scope()`.
    multi_worker_model = classifier_def.build_and_compile()
